In [13]:
import numpy as np
import pandas as pd
import locale

locale.setlocale(locale.LC_TIME, "de_DE")

%store -r columnNames

#pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199


# pd.describe_option('display')

# Analyze Data

In [14]:
surveyData=pd.read_csv('justCompletedAttempts.csv', sep=';')
print("Anzahl an kompletten Daten:", len(surveyData))

# Add extra fields for comparison
surveyData['duartion-date-based'] = surveyData.apply(lambda row: pd.to_datetime(row['date-last'])-pd.to_datetime(row['date-start']), axis=1)
surveyData['duartion-date-based-in-seconds'] = surveyData.apply(lambda row: row['duartion-date-based'].total_seconds(), axis=1)
surveyData['duartion-delta-all-vs-start-end'] = surveyData.apply(lambda row: row['duration-over-all']-row['duartion-date-based-in-seconds'], axis=1)


Anzahl an kompletten Daten: 203


## Time Delta

In [15]:

# Check time delta
greatDelta = 10 
extremeDelta = 20
print()
print(f"Overall länger",len(surveyData.loc[surveyData['duartion-delta-all-vs-start-end'] >= 0]))
print(f"Overall sehr viel länger, AKA Start-End schneller >{greatDelta} Sek. schneller:",len(surveyData.loc[surveyData['duartion-delta-all-vs-start-end'] >= greatDelta]))
print(f"Overall extrem länger, AKA Start-End schneller >{extremeDelta} Sek. schneller:",len(surveyData.loc[surveyData['duartion-delta-all-vs-start-end'] >= extremeDelta]))
print()
print(f"Start-End länger",len(surveyData.loc[surveyData['duartion-delta-all-vs-start-end'] < 0]))
print(f"Start-End sehr viel länger, AKA Overall >{greatDelta} Sek. schneller:",len(surveyData.loc[surveyData['duartion-delta-all-vs-start-end'] < -greatDelta]))
print(f"Start-End extrem länger, AKA Overall >{extremeDelta} Sek. schneller:",len(surveyData.loc[surveyData['duartion-delta-all-vs-start-end'] < -extremeDelta]))

# surveyData[['id', 'duration-over-all', 'duartion-date-based-in-seconds', 'duartion-delta-all-vs-start-end', 'duartion-date-based']]


Overall länger 134
Overall sehr viel länger, AKA Start-End schneller >10 Sek. schneller: 10
Overall extrem länger, AKA Start-End schneller >20 Sek. schneller: 9

Start-End länger 69
Start-End sehr viel länger, AKA Overall >10 Sek. schneller: 20
Start-End extrem länger, AKA Overall >20 Sek. schneller: 16


## Building Groups

In [16]:
# Build groups depending on what viz participant had to solve first

listFirstParticipants=surveyData[surveyData['list-question-number']==6]

chordFirstParticipants=surveyData[surveyData['chord-question-number']==6]

mapFirstParticipants=surveyData[surveyData['map-question-number']==6]

print("Liste zu erst", len(listFirstParticipants))
print("Chord zu erst", len(chordFirstParticipants))
print("Map zu erst", len(mapFirstParticipants))

Liste zu erst 52
Chord zu erst 78
Map zu erst 73


### Define Evaluation Columns

In [17]:
# Build evaluation columns 

evalColumns = [
    'id', 
    'date-start', 
    'date-last',
    'duration-over-all',
    'duration-group-tasks-intro',
    'duration-group-list-task',
    'duration-group-chord-task',
    'duration-group-map-task',
    'age', 
    'sex', 
    'internet-level', 
    'mood-start', 
    'mood-end', 
    'preference-chord-or-map-over-list', 
    'highest-preferenced-viz', 
    'preference-reason', 
    'qualitative-questions-pros-cons-chord', 
    'qualitative-questions-pros-cons-map', 
    'qualitative-questions-pros-cons-list'
]

## Write Out Data

In [19]:
# Write data to analyze quality feedback

for index, row in surveyData.iterrows():
    # Create standard line
    print("Meine Ausgabe")


Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine Ausgabe
Meine 